In [1]:
import os
import time

import torch.nn as nn
import torch

import torch.optim as optim

from torchvision import datasets,models,transforms

import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_dir = "/content/drive/MyDrive/paper/data/hym_data"

batch_size = 64
num_workers = 2

data_transformers = {
    'train' : transforms.Compose(
        [
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.490,0.449,0.411],[0.231,0.221,0.230])
        ]
    ),
    'val' : transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.490,0.449,0.411],[0.231,0.221,0.230])
        ]
    )
}

img_data = {
    k : datasets.ImageFolder(os.path.join(data_dir,k),data_transformers[k]) for k in ['train','val']
}
dloaders = {
    k: torch.utils.data.DataLoader(
        img_data[k], batch_size=batch_size, shuffle=True, num_workers=num_workers
    )
    for k in ['train', 'val']
}
dset_sizes = {x:len(img_data[x]) for x in ['train','val']}
classes = img_data['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
import torch
import torch.nn as nn

cfg = {
    'vgg16' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256,      'M', 512, 512, 512,      'M', 512, 512, 512,      'M'],
    'vgg19' : [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}

class VGG(nn.Module):

    def __init__(self, features, num_classes=100):
        super().__init__()
        self.features = features

        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        output = self.features(x)
        output = output.view(output.size()[0], -1)
        output = self.classifier(output)

        return output

def make_layers(cfg):
    layers = []

    input_channel = 3
    for l in cfg:
        if l == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            continue

        layers += [nn.Conv2d(input_channel, l, kernel_size=3, padding=1)]

       
        layers += [nn.BatchNorm2d(l)]

        layers += [nn.ReLU(inplace=True)]
        input_channel = l

    return nn.Sequential(*layers)


def vgg16(num=100):
    return VGG(make_layers(cfg['vgg16']),num_classes=num)

def vgg19(num=100):
    return VGG(make_layers(cfg['vgg19']),num_classes=num)



In [5]:
def train(model,loss_func,optimizer , epochs = 10):

    start = time.time()

    accuracy= 0.0

    for e in range(epochs):
        print(f'Epoch number {e+1}/{epochs}')
        print('=' * 20)

        for dset in ['train','val']:
            if dset == 'train':
                model.train()
            else:
                model.eval()

            loss =0.0
            suc = 0

            for img,lab in dloaders[dset]:
                img = img.to(device)
                lab = lab.to(device)
                
                optimizer.zero_grad()

                with torch.set_grad_enabled(dset=='train'):
                    ops = model(img)
                    _,preds = torch.max(ops,1)
                    loss_cur = loss_func(ops,lab)

                    if dset == 'train':
                        loss_cur.backward()
                        optimizer.step()

                loss += loss_cur.item() * img.size(0)
                suc += torch.sum(preds == lab.data)

            loss_epoch = loss / dset_sizes[dset]
            accuracy_epoch = suc.double() / dset_sizes[dset]

            print(f'{dset} loss in this epoch: {loss_epoch}, accuracy in this epoch: {accuracy_epoch}')
            if dset == 'val' and accuracy_epoch > accuracy:
                accuracy = accuracy_epoch    
    time_delta = time.time() - start
    print(f'Training finished in {time_delta // 60}mins {time_delta % 60}secs')
    print(f'Best validation set accuracy: {accuracy}')

    return model

In [6]:
model = vgg19(2)
if torch.cuda.is_available():
  model = model.cuda()
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [7]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
pretrained_model = train(model, loss_func, optimizer, epochs=5)

Epoch number 1/5
train loss in this epoch: 2.2518058018606215, accuracy in this epoch: 0.5122950819672132
val loss in this epoch: 0.6964161540168563, accuracy in this epoch: 0.4575163398692811
Epoch number 2/5
train loss in this epoch: 1.2116489117262794, accuracy in this epoch: 0.5245901639344263
val loss in this epoch: 0.7279671769516141, accuracy in this epoch: 0.4575163398692811
Epoch number 3/5
train loss in this epoch: 0.6768076595712881, accuracy in this epoch: 0.5860655737704918
val loss in this epoch: 0.6923533089799818, accuracy in this epoch: 0.542483660130719
Epoch number 4/5
train loss in this epoch: 0.8025368129620787, accuracy in this epoch: 0.49180327868852464
val loss in this epoch: 0.6896297238231485, accuracy in this epoch: 0.542483660130719
Epoch number 5/5
train loss in this epoch: 0.6735572140724932, accuracy in this epoch: 0.5245901639344263
val loss in this epoch: 0.7134709790641186, accuracy in this epoch: 0.4575163398692811
Training finished in 1.0mins 10.7404

In [9]:
def test(model=pretrained_model):

  correct_pred = {classname: 0 for classname in classes}
  total_pred = {classname: 0 for classname in classes}


  with torch.no_grad():
      for images, labels in dloaders['val']:


          images = images.to(device)
          labels = labels.to(device)
        
          outputs = model(images)
          _, predictions = torch.max(outputs, 1)

          for label, prediction in zip(labels, predictions):
              if label == prediction:
                  correct_pred[classes[label]] += 1
              total_pred[classes[label]] += 1



  for classname, correct_count in correct_pred.items():
      accuracy = 100 * float(correct_count) / total_pred[classname]
      print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [10]:
test(pretrained_model)

Accuracy for class: ants  is 100.0 %
Accuracy for class: bees  is 0.0 %
